In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from torchvision import datasets, transforms

In [2]:
from data_loader import get_loader
import helper

In [3]:
data_loader = get_loader('CelebA', 'train', 16, 64, 4, True)
data_loader = iter(data_loader)

Found 999 images in subfolders of: CelebA/splits/train


In [4]:
mb_size = 16
z_dim = 128
X_dim = 64
y_dim = 64
h_dim = 128
cnt = 0
d_step = 3
lr = 1e-3
m = 5
lam = 1e-3
k = 0
gamma = 0.5

In [5]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(32, 48, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv4 = nn.Conv2d(48, 64, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=0, padding=0, ceil_mode=True)

        # Starting image is of size 64, 4 times maxpooled down to 8.
        # Map to the embedding size.
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        
        # Reconstruct the image from the embedding.
        self.fc2 = nn.Linear(128, 3 * 64 * 64)
        
        # Activation function
        self.activation = F.relu
        self.activation = lambda x: x
        
    def forward(self, x):
        x = x.view(-1, 3 , 64, 64)
        x = self.activation(self.maxpool(self.conv1(x)))
        x = self.activation(self.maxpool(self.conv2(x)))
        x = self.activation(self.maxpool(self.conv3(x)))
        x = self.activation(self.conv4(x))

        x = x.view(-1, 64 * 8 * 8)
        
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))

        return x
    
D_ = Discriminator()

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        # Map the embedding to an 8x8 image with 16 channels
        self.fc1 = nn.Linear(128, 8 * 8 * 16)

        self.conv = nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv_f = nn.Conv2d(16, 3, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.upsample = nn.UpsamplingNearest2d(scale_factor=2)
        
        # Activation function
        self.activation = F.relu
        self.activation = lambda x: x
        
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = x.view(-1, 16, 8, 8)
        
        x = self.activation(self.upsample(self.conv(self.conv(x))))
        x = self.activation(self.upsample(self.conv(self.conv(x))))
        x = self.activation(self.upsample(self.conv(self.conv(x))))

        x = self.activation(self.conv_f(x))
        x = x.view(-1, 3 * 64 * 64)
        
        return x
    
G = Generator()

In [7]:
# D is an autoencoder, approximating Gaussian
def D(X):
    X_recon = D_(X)
    # helper.tensor_imshow(X.data[0])
    # helper.tensor_imshow(X_recon.data[0].view(3, 64, 64))
    
    # Use Laplace MLE as in the paper
    return torch.mean(torch.sum(torch.abs(X - X_recon), 1))

def reset_grad():
    G.zero_grad()
    D_.zero_grad()

In [8]:
G_solver = optim.Adam(G.parameters(), lr=lr)
D_solver = optim.Adam(D_.parameters(), lr=lr)

In [10]:
# Train
train_iter = 2
for it in range(train_iter):
    # Sample data
    X, _ = next(data_loader)
    X = Variable(X)

    # Dicriminator
    # Create a random tensor of size [mini_batch x embedding_vector_dim]
    z_D = Variable(torch.randn(mb_size, z_dim))

    D_loss = D(X) - k * D(G(z_D))

    D_loss.backward()
    D_solver.step()
    reset_grad()

    # Generator
    z_G = Variable(torch.randn(mb_size, z_dim))

    G_loss = D(G(z_G))

    G_loss.backward()
    G_solver.step()
    reset_grad()

    # Update k, the equlibrium
    k = k + lam * (gamma*D(X) - D(G(z_G)))
    k = k.data[0]  # k is variable, so unvariable it so that no gradient prop.

    # Print and plot every now and then
    if it % 1000 == 0:
        measure = D(X) + torch.abs(gamma*D(X) - D(G(z_G)))

        print('Iter-{}; Convergence measure: {:.4}'
              .format(it, measure.data[0]))

        samples = G(z_G).data.numpy()[:16]

        fig = plt.figure(figsize=(4, 4))
        gs = gridspec.GridSpec(4, 4)
        gs.update(wspace=0.05, hspace=0.05)

        for i, sample in enumerate(samples):
            ax = plt.subplot(gs[i])
            plt.axis('off')
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_aspect('equal')
            plt.imshow(sample.reshape(64, 64))

        if not os.path.exists('out/'):
            os.makedirs('out/')

        plt.savefig('out/{}.png'.format(str(cnt).zfill(3)), bbox_inches='tight')
        cnt += 1
        plt.close(fig)


torch.Size([16, 12288])
torch.Size([16, 12288])


RuntimeError: matrices expected, got 4D, 2D tensors at /Users/adithya/GradSchool/DL/pytorch/torch/lib/TH/generic/THTensorMath.c:857